In [196]:
import pandas as pd
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
from openai import OpenAI

In [197]:
df_rekt = pd.read_csv('../datasets/web3isgoinggreat_dataset.csv', index_col=0)

In [198]:
df_rekt = df_rekt.dropna()

In [199]:
df_rekt = df_rekt[['Hack' in i for i  in  df_rekt.tags]]

In [200]:
df_rekt

,title,date,summary,tags
0,"""Peripheral"" Aave smart contract hacked for $5...","August 28, 2024","The popular defi lending platform, Aave, suffe...",Hack or scam
3,"Brothers charged by SEC for $60 million ""crypt...","August 26, 2024",Brothers Jonathan and Tanner Adam were charged...,"Hack or scam, Law"
5,Users suffer losses after Polygon Discord hack,"August 24, 2024","Some fans of the Polygon blockchain, or those ...",Hack or scam
6,"McDonald's Instagram hacked, hackers claim $70...","August 21, 2024","McDonald's Instagram account, as well as the T...",Hack or scam
7,Crypto holder loses over $55 million to appare...,"August 20, 2024",Someone holding almost $55.5 million in the DA...,Hack or scam
...,...,...,...,...
559,Sentiment protocol hacked for almost $1 million,"April 4, 2023",The Sentiment liquidity protocol on the Arbitr...,Hack or scam
562,Over $25 million taken from an MEV bot by mali...,"April 3, 2023",It's a dog-eat dog-world in the crypto univers...,Hack or scam
565,Allbridge cross-chain bridge exploited for aro...,"April 1, 2023",The Allbridge cross-chain bridge project was e...,"Bug, Hack or scam"
567,"Arbitrum airdrop plagued by downtime, bugs, an...","March 31, 2023",A token airdrop from the popular Arbitrum Ethe...,"Hack or scam, Hmm"


In [201]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

/home/seacevedo/.local/share/virtualenvs/rekt_qa_system-pBD0ZnT5/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [202]:
def generate_embeddings(row):
    title = row['title']
    date = row['date']
    tags = row['tags']
    if pd.isna(tags):
        tags = ''
    summary = row['summary']
    title_summary = title + ' ' + summary

    row['title_vec'] = model.encode(title)
    row['date_vec'] = model.encode(date)
    row['tags_vec'] = model.encode(tags)
    row['summary_vec'] = model.encode(summary)
    row['all_vec'] = model.encode(title_summary)

    return row

In [203]:
df_rekt = df_rekt.apply(generate_embeddings, axis=1)

In [204]:
df_rekt

,title,date,summary,tags,title_vec,date_vec,tags_vec,summary_vec,all_vec
0,"""Peripheral"" Aave smart contract hacked for $5...","August 28, 2024","The popular defi lending platform, Aave, suffe...",Hack or scam,"[-0.035581745, -0.007754507, 0.05457383, -0.06...","[0.0045564254, 0.021765376, 0.03966033, 0.0135...","[-0.10653031, -0.014528473, -0.056755323, -0.0...","[-0.115749426, -0.015938954, -0.009345272, -0....","[-0.08428081, -0.01666155, 0.025461914, -0.071..."
3,"Brothers charged by SEC for $60 million ""crypt...","August 26, 2024",Brothers Jonathan and Tanner Adam were charged...,"Hack or scam, Law","[-0.039006256, -0.0058030416, -0.105479404, -0...","[0.012031223, 0.032236855, 0.036864, 0.0274487...","[-0.07951577, 0.0018831257, -0.062856875, -0.0...","[-0.035585437, 0.0775032, -0.095088296, -0.037...","[-0.03972881, 0.03715992, -0.11536511, -0.0306..."
5,Users suffer losses after Polygon Discord hack,"August 24, 2024","Some fans of the Polygon blockchain, or those ...",Hack or scam,"[0.094478205, 0.0002943923, 0.10371892, 0.0383...","[0.009936101, 0.024253476, 0.045322254, 0.0187...","[-0.10653031, -0.014528473, -0.056755323, -0.0...","[0.06251927, -0.044542138, 0.06750497, 0.02732...","[0.08304511, -0.034339305, 0.0849363, 0.036773..."
6,"McDonald's Instagram hacked, hackers claim $70...","August 21, 2024","McDonald's Instagram account, as well as the T...",Hack or scam,"[-0.051038146, -0.057920847, 0.04719373, -0.01...","[0.01828971, 0.036256123, 0.04899738, 0.019191...","[-0.10653031, -0.014528473, -0.056755323, -0.0...","[-0.05843942, -0.029506342, 0.081961945, 0.030...","[-0.064780414, -0.03958614, 0.073366374, 0.018..."
7,Crypto holder loses over $55 million to appare...,"August 20, 2024",Someone holding almost $55.5 million in the DA...,Hack or scam,"[0.06181611, 0.07224824, 0.021658946, -0.02691...","[0.009036094, 0.03263282, 0.06939553, -0.01048...","[-0.10653031, -0.014528473, -0.056755323, -0.0...","[0.01119119, 0.10501865, -0.034607317, 0.02776...","[0.046877768, 0.11102805, -0.018116426, 0.0134..."
...,...,...,...,...,...,...,...,...,...
559,Sentiment protocol hacked for almost $1 million,"April 4, 2023",The Sentiment liquidity protocol on the Arbitr...,Hack or scam,"[0.036276255, 0.011144656, 0.052093152, -0.036...","[-0.062381897, -0.025215836, 0.041302457, 0.00...","[-0.10653031, -0.014528473, -0.056755323, -0.0...","[0.029724017, 0.08864613, 0.04146631, 0.013756...","[0.032532487, 0.08176808, 0.042567898, 0.00066..."
562,Over $25 million taken from an MEV bot by mali...,"April 3, 2023",It's a dog-eat dog-world in the crypto univers...,Hack or scam,"[-0.028061556, 0.015294973, -0.028831054, -0.0...","[-0.08900714, -0.023725204, 0.036874328, 0.001...","[-0.10653031, -0.014528473, -0.056755323, -0.0...","[-0.07917025, -0.01496485, -0.029967694, -0.02...","[-0.05319384, 0.009125692, -0.028843747, -0.03..."
565,Allbridge cross-chain bridge exploited for aro...,"April 1, 2023",The Allbridge cross-chain bridge project was e...,"Bug, Hack or scam","[-0.027797082, 0.03515872, 0.043409184, 0.0070...","[-0.07205014, -0.011304095, 0.04496276, 0.0055...","[-0.13383146, -0.04394348, -0.022239096, -0.02...","[-0.058315013, 0.036641665, 0.054380145, -0.02...","[-0.05880784, 0.038473934, 0.05723856, -0.0127..."
567,"Arbitrum airdrop plagued by downtime, bugs, an...","March 31, 2023",A token airdrop from the popular Arbitrum Ethe...,"Hack or scam, Hmm","[0.0061603463, 0.0041198605, 0.0077677504, 0.0...","[-0.06676466, -0.0007074958, 0.04269806, -0.02...","[-0.10288791, 0.019725911, -0.04168526, -0.037...","[-0.003626018, 0.0074737277, -0.005211955, 0.0...","[-0.0019273496, 0.018207636, 5.0280785e-05, 0...."


## Indexing

In [205]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "date": {"type": "text"},
            "summary": {"type": "text"},
            "tags": {"type": "text"},
            "title_vec": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "date_vec": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "tags_vec": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "summary_vec": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "all_vec": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "rekt-knowledgebase"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'rekt-knowledgebase'})

In [206]:
rekt_docs = df_rekt.to_json(orient="records")

In [207]:
from json import loads, dumps
parsed = loads(rekt_docs)

In [208]:
for doc in tqdm(parsed):
    es_client.index(index=index_name, document=doc)
    #print(doc)

100%|█████████████████████████████████████████████████████████████████████████████████| 261/261 [00:02<00:00, 88.07it/s]


## Search

In [209]:
query = 'What hacks occured on August 15, 2023?'

In [210]:
v_q = model.encode(query)

In [211]:
knn_query = {
    "field": "all_vec",
    "query_vector": v_q,
    "k": 5,
    "num_candidates": 10000,
    "boost": 0.5
}

In [212]:
keyword_query = {
    "bool": {
        "must": {
            "multi_match": {
                "query": query,
                "fields": ["title", "date^2", "summary", "tags"],
                "type": "best_fields",
                "boost": 0.5,
            }
        }
    }
}

In [213]:
response = es_client.search(
    index=index_name,
    query=keyword_query,
    knn=knn_query,
    size=5
)

In [214]:
def elastic_search(query):
    v_q = model.encode(query)
    
    knn_query = {
        "field": "all_vec",
        "query_vector": v_q,
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["title", "date^2", "summary", "tags"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            }
        }
    }

    response = es_client.search(
        index=index_name,
        query=keyword_query,
        knn=knn_query,
        size=5
    )

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [215]:
results = elastic_search(query)

## LLM

In [275]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [276]:
def llm(prompt, model='phi3', temperature=0.0):
    response = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [277]:
def build_prompt(query, search_results):

    prompt_template = """
    You're a assistant that informs the user on the latest cryptocurrency hacks and exploits. Answer the QUESTION based on the CONTEXT from our crytocurrency hacks and exploits database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    entry_template = """
    title: {title}
    date: {date}
    summary: {summary}
    tags: {tags}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [278]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [279]:
query_1 = 'What hacks occured on August 1, 2024?'

In [280]:
rag(query)

'On August 15, 2023, there were no reported hacks specifically occurring to the information provided in our database of cryptocurrency hacks and exploits on that date. However, a hack did occur at Remitano for $2.7 million earlier on September 15, 2023.'